In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from astropy.io import fits
from lmfit import Model, Parameters, minimize
import scipy.ndimage
import copy
from scipy.ndimage.filters import gaussian_filter
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import Cutout2D

In [ ]:
'''
This is where we first make a cut out of the region of interest and look at the polarisation data. 
Make sure that everything is on the same grid, that the polaristion angle is going counter clockwise from north,
that all the necessary analysis is applied to the PA data beofre starting the process.
Do not forget the velocity dispersion data in km/s

This process will require some tweaking so make sure that you read every part. Good luck!!
'''

#First we set a few constants that we will use. 

#This is the depth of at which we are looking. (2*z in the model) this value is in pixels Make sure that it is large enough but not too big
depth = 26

xcenter=18.63 #location of the x value of the centre of the image (degrees)
ycenter=-0.065 #location of the y value of the centre of the image (degrees)
coords = SkyCoord(l=xcenter*u.degree, b=ycenter*u.degree, frame='galactic', unit= (u.deg) ) #This is for gala

width= 35
height= 20
X = width
Y = height



column_density_file = '/home/jessym/harvard/fil_5/fits/HIGAL_L_Fil5_Ngas.fits'
column_fits = fits.open(column_density_file)
column_data = column_fits[0].data
column_header = column_fits[0].header
column_world = WCS(column_header)
column_centre = column_world.world_to_pixel(coords)



BAngle_file = '/home/jessym/harvard/fil_5/fits/Fil5_BAngle_reprojectGal.fits'
BAngle_fits = fits.open(BAngle_file)
BAngle_data = BAngle_fits[0].data
BAngle_header = BAngle_fits[0].header
p = fits.open('/home/jessym/harvard/fil_5/fits/Fil5_Percent_debiased_reprojectGal.fits')[0].data
p_e = fits.open('/home/jessym/harvard/fil_5/fits/Fil5_Percent_e_reprojectGal.fits')[0].data
sn = p/p_e
(ys,xs)=p.shape

for y in range(0,ys,1):
    for x in range(0,xs,1):
        if sn[y,x] >= 3 and p[y,x] != 'nan':
            continue
        else:
            BAngle_data[y,x] = np.nan

#BAngle_world = WCS(BAngle_header)
#BAngle_centre = BAngle_world.world_to_pixel(coords)

from reproject import reproject_exact
widths = fits.open('/home/jessym/harvard/fil_5/fits/line_analysis/grs_fit/13CO_cube_single_width.fits')[0]
widths_reproj = reproject_exact(widths, column_header, return_footprint = False)
widths_cut = Cutout2D(widths_reproj, column_centre, (height, width)).data
B_reproj = reproject_exact((BAngle_data, BAngle_header), column_header, return_footprint = False)
BAngle_data_cut = Cutout2D(B_reproj, column_centre, (height, width)).data
#BAngle_data_cut = np.array(itertools.islice(BAngle_data_cut, 0, None, 3))
data = Cutout2D(column_data, column_centre, (height, width)).data
med = np.median(data)

emptymap = np.copy(data)
emptymap[:] = np.nan
mask = emptymap.copy()

for i in range(np.shape(data)[0]):
    for j in range(np.shape(data)[1]):
        if 2e22 <= data[i,j]: # and not np.isnan(widths_cut[i,j])
            mask[i,j] = 1
        else:
            mask[i,j] = np.nan

BAngle_data_cut = BAngle_data_cut*mask

Bdir_x = -np.sin(np.array(BAngle_data_cut)*np.pi/180)
Bdir_y = np.cos(np.array(BAngle_data_cut)*np.pi/180)
save = []


norm_data = data/med
plt.figure(figsize=(6,6))
plt.title('Ngas Data')
plt.imshow(norm_data, origin='lower')
contour_levels = np.arange(np.nanmin(norm_data), np.nanmax(norm_data), (np.nanmax(norm_data)-np.nanmin(norm_data))/6)
plt.colorbar(location='bottom', ticks = contour_levels)
#cont = gaussian_filter(data, 1)
CS = plt.contour(norm_data, levels=contour_levels, colors = 'white', alpha = 0.7)
plt.savefig('send_phil/Ngas_data.pdf')
plt.show()
print(np.min(norm_data), np.max(norm_data))
print(contour_levels)
print(43.51/14)
print(np.shape(BAngle_data_cut))
print(np.shape(Bdir_x))

x = np.arange(0,np.shape(BAngle_data_cut)[1],1)
y = np.arange(0,np.shape(BAngle_data_cut)[0],1)
xarr, yarr = np.meshgrid(x,y)
plt.figure(figsize=(14,4))
plt.subplot(1,2,1)
#plt.figure(figsize=(6,3.4))
#plt.imshow(BAngle_data_cut, origin='lower')
#plt.colorbar(location = 'bottom')
plt.streamplot(xarr,yarr,Bdir_x,Bdir_y, density=1, linewidth=None, color='k')
plt.title('HAWC+ polarization (stream function)')
#plt.show()

plt.subplot(1,2,2)
data_angle = np.radians(BAngle_data_cut)
(ys,xs)=data_angle.shape
step=1
scale=step
#plt.figure(figsize=(6,3.4))
for y in range(0,ys,step):
    for x in range(0,xs,step):
        r=scale*0.5
        x1_d=x-r*np.sin(data_angle[y,x])   
        y1_d=y+r*np.cos(data_angle[y,x])
        x2_d=x+r*np.sin(data_angle[y,x])
        y2_d=y-r*np.cos(data_angle[y,x])

        plt.plot([x1_d,x2_d],[y1_d,y2_d], color = 'k',  linewidth=3) 

        #line=np.array([[x1_d,x2_d],[y1_d,y2_d]])
        #list_y_s.append(line)
#gc.show_lines(list_r_s,linewidth=2,color='red')
plt.title('HAWC+ polarization (segments)')
plt.savefig('send_phil/HAWC+_polarization.pdf')
plt.show()